# Prepare the nodes

In [1]:
data = {'nodes':[], 'links':[]}

In [2]:
import pandas as pd
gbc_ff = pd.read_csv('gbc_ff.csv', dtype={'PMID':'int64'})
gbc_ff = gbc_ff[['PMID', 'Title']]
print(gbc_ff.dtypes)
gbc_ff = {'PMID':gbc_ff['PMID'].tolist(), 'Title':gbc_ff['Title'].tolist()}

PMID      int64
Title    object
dtype: object


In [3]:
pmid2ind = {}
for i, pmid in enumerate(gbc_ff['PMID']):
    pmid2ind.update({pmid: i})

In [38]:
print(gbc_ff)

{'PMID': [10731134, 10778742, 11030340, 11070064, 11099034], 'Title': ['Comparative genomics of the eukaryotes.', 'Mutational accessibility of essential genes on chromosome I(left) in Caenorhabditis elegans.', 'A global profile of germline gene expression in C. elegans.', 'Lateral gene transfer and metabolic adaptation in the human parasite Trichomonas vaginalis.', 'Functional genomic analysis of cell division in C. elegans using RNAi of genes on chromosome III.']}


In [4]:
for title in gbc_ff['Title']:
    data['nodes'].append({'name':title, 'group':1})

# Get the links

In [5]:
import requests
import time
import json

In [6]:
def pmid2pmids(pmid):
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&'+\
          f'id={pmid}' + '&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08'
    try:
        rs = requests.get(url)
        time.sleep(0.1)
        rs_dict = json.loads(rs.text)
        item = rs_dict['linksets'][0]
        if 'linksetdbs' in item:
            pmids = item['linksetdbs'][0]['links'] # returned pmids is a list of integer
            return pmids
        else:
            return None
    
    except requests.exceptions.RequestException as e:
        print(f'requests.get({url}) results in this error:')
        print(e)
        return None


In [9]:
print(pmid2pmids(22777646))

[22258609, 22168571, 22129982, 21516124, 21185705, 21119631, 20843320, 20688898, 20647327, 20570920, 20550684, 20479776, 20376786, 20372786, 20363604, 20007894, 19900289, 19887543, 19810033, 19582872, 19304016, 19287097, 18671839, 18281385, 17543572, 17406949, 16931107, 16407131, 16267003, 16203864, 15756016, 15297404, 15032581, 14688310, 12865923, 12610629, 12559043, 11900991, 11694782, 11323675, 10716946, 10528190, 9064324, 1003551]


In [7]:
print(len(data['nodes']))

2684


In [8]:
gbc_pmids_set = set(gbc_ff['PMID'])
i = 0
for pmid in gbc_ff['PMID']:
    i+=1
    if i%10==0:
        print(i)
    ref_pmids = pmid2pmids(pmid)
    if ref_pmids:
        print('has ref')
        for rp in ref_pmids:
            if rp in gbc_pmids_set:
                data['links'].append({'source':pmid2ind[pmid], 'target':pmid2ind[rp], 'value':1})

has ref
10
has ref
has ref
has ref
has ref
has ref
has ref
20
has ref
has ref
has ref
30
has ref
has ref
40
has ref
has ref
has ref
has ref
has ref
50
has ref
has ref
has ref
60
70
has ref
has ref
has ref
has ref
has ref
80
has ref
has ref
has ref
90
has ref
has ref
has ref
has ref
has ref
has ref
100
has ref
has ref
has ref
110
has ref
has ref
has ref
has ref
120
has ref
has ref
has ref
has ref
has ref
has ref
130
has ref
has ref
has ref
has ref
140
has ref
has ref
has ref
150
has ref
has ref
has ref
160
has ref
has ref
170
has ref
has ref
has ref
180
has ref
has ref
has ref
190
has ref
200
has ref
has ref
has ref
210
has ref
has ref
has ref
has ref
220
has ref
has ref
has ref
has ref
has ref
has ref
230
has ref
has ref
has ref
has ref
240
has ref
250
has ref
260
has ref
has ref
has ref
has ref
has ref
270
has ref
has ref
has ref
has ref
280
has ref
290
has ref
has ref
300
has ref
has ref
has ref
310
has ref
has ref
320
has ref
has ref
has ref
has ref
has ref
has ref
330
has ref
has r

has ref
1970
has ref
has ref
has ref
has ref
has ref
has ref
1980
has ref
has ref
has ref
has ref
1990
has ref
has ref
has ref
has ref
has ref
2000
has ref
has ref
has ref
has ref
2010
has ref
has ref
has ref
has ref
has ref
has ref
has ref
2020
has ref
has ref
has ref
has ref
has ref
2030
has ref
has ref
has ref
has ref
2040
has ref
has ref
has ref
has ref
has ref
has ref
has ref
2050
has ref
has ref
has ref
has ref
has ref
2060
has ref
has ref
has ref
has ref
has ref
has ref
2070
has ref
has ref
has ref
2080
has ref
has ref
has ref
has ref
has ref
has ref
2090
has ref
has ref
has ref
has ref
has ref
has ref
has ref
has ref
has ref
2100
has ref
has ref
has ref
has ref
has ref
has ref
2110
has ref
has ref
has ref
has ref
has ref
2120
has ref
has ref
has ref
has ref
has ref
has ref
2130
has ref
has ref
has ref
has ref
has ref
has ref
2140
has ref
has ref
has ref
has ref
has ref
has ref
has ref
has ref
2150
has ref
has ref
has ref
has ref
has ref
2160
has ref
has ref
has ref
has ref
2170

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import pickle
with open('gbc_graph_data.pickle', 'wb') as f:
    pickle.dump(data, f)

# Get rid of the items whose degree is 0

In [35]:
import pickle
with open('gbc_graph_data.pickle', 'rb') as f:
    data = pickle.load(f)
for i, node in enumerate(data['nodes']):
    node.update({'in-deg':0, 'out-deg':0, 'ori_ind':i})
for link in data['links']:
    src = link['source']
    dest = link['target']
    data['nodes'][src]['out-deg']+=1
    data['nodes'][dest]['in-deg']+=1
    

In [38]:
len(data['nodes'])

2684

In [40]:
data['nodes'] = [x for x in data['nodes'] if x['in-deg']+x['out-deg']>0 ] 
oriInd2Ind = {}
for i, node in enumerate(data['nodes']):
    oriInd2Ind.update({node['ori_ind']:i})
for link in data['links']:
    link['source'] = oriInd2Ind[link['source']]
    link['target'] = oriInd2Ind[link['target']]

In [41]:
len(data['nodes'])

1544

In [42]:
import pickle
with open('gbc_graph_data_degree1plus.pickle', 'wb') as f:
    pickle.dump(data, f)

# Draw the map

In [43]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [44]:
labels=[]
group=[]
size=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])
    size.append(node['in-deg'])

In [45]:
import igraph as ig
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)
layt = G.layout('kk', dim=3)
# layt is a list of three elements lists (the coordinates of nodes)
# layt = [[x1, y1, z1], [x2, y2, z2], ...]

In [46]:
import pickle
with open('layout_with_degree1plus.pickle', 'wb') as f:
    pickle.dump(layt, f)

In [47]:
#N = len(data['nodes'])
N = len(layt)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None] 

In [50]:
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=size,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Reference Relation of Genome BC funded papers",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [51]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)